## Goal: 載入儲存的efficientnet模型，對公開測試資料進行預測，並將結果上傳到平台，查看模型表現分數

In [1]:
from itertools import islice
from PIL import Image
from sklearn import preprocessing
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.io import read_image
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.transforms import ToTensor
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pandas as pd
import torch
import torchvision
import time
import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from tqdm import tqdm

prjt_path = "/content/gdrive/MyDrive/learning_AIdea2022"
# train_data_path = str(prjt_path + r"/train_data")  # 競賽給的資料集
test_data_path = str(prjt_path + r"/all_private_test")  # 競賽給的公開測試資料集
output_path = str(prjt_path + r"/output")

In [2]:
import torch
print(torch.cuda.is_available())

device = (torch.device('cuda') if torch.cuda.is_available()
         else torch.device('cpu'))
print(f"Training on device {device}.")

True
Training on device cuda.


In [3]:
# Data Path
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

# cd /content/gdrive/MyDrive/learning_AIdea2022/private_test
# train_data_path = "/content/gdrive/MyDrive/learning_AIdea2022/private_test"
# output_path = str("/content/gdrive/MyDrive/learning_AIdea2022/all_private_test")

Mounted at /content/gdrive


## 1.Load the Pretrained model

In [4]:
efficientnet_path_model = str(output_path + "/ALL_efficientnet_model.pkl")
model_efficientnet = torch.load(efficientnet_path_model)
model_efficientnet.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

## 2.Load test data (載入競賽平台提供的公開測試資料集)

In [5]:
ALL_test_data_path = test_data_path
print('Total test images:', len(os.listdir(ALL_test_data_path)))

Total test images: 1366


In [7]:
# use_save_model_to_predict

In [8]:
print(len(os.listdir(ALL_test_data_path)))
print(type(os.listdir(ALL_test_data_path)))

1366
<class 'list'>


In [9]:
# # Ref: Crop_1st.ipynb
# for img_name in os.listdir(ALL_test_data_path):
#     #print(img_name)
#     img_data = cv2.imread(test_data_path + "\\" + img_name)
#     print(type(img_data))
#     #img_shape = img_data.shape
#     #print(img_shape.shape[0])
#     #print(img_shape)

### Debug:
* ERROR: PyTorch - Getting the 'TypeError: pic should be PIL Image or ndarray. Got <class 'numpy.ndarray'>'
* Sol: `transforms.ToPILImage()`
* [Stackoverflow](https://stackoverflow.com/questions/56741108/pytorch-getting-the-typeerror-pic-should-be-pil-image-or-ndarray-got-class)

In [10]:
t = transforms.Compose([
    transforms.ToPILImage(),  # first convert the numpy array to a PIL Image
    transforms.ToTensor(),  # you can add other transformations in this list
    transforms.Resize(size=(255, 255))
])

In [11]:
class CustomImageDataset_NOT_label(Dataset):
    def __init__(self, transform=None, target_transform=None):
        self.img_name =  os.listdir(ALL_test_data_path)
        #print(self.img_name[100])
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, idx):
        #print(self.img_name[100])
        img_path = os.path.join(test_data_path, str(self.img_name[idx]))
        img_specific_name = self.img_name[idx]
        image = read_image(img_path)
        if self.transform:
            image = self.transform(image)
        return img_specific_name, image

In [12]:
test_dataset = CustomImageDataset_NOT_label(transform=t)  # same size to each img
# print(test_dataset[1][1])  # 第一張圖片的 tensor
# print(test_dataset[1][0])  # 第一張圖片的 檔案名稱: 0006061f-abda-4e2b-a07f-506ce19a0efc.jpg
# print(len(test_dataset))

In [13]:
batch_size = 10 # 一個batch有10張圖片
# val_size, test_size = 0.1, 0.1  # train:val:test=0.8:0.1:0.1
# shuffle_dataset = True
# random_seed= 42
# transform = transforms.Compose([
#     transforms.ToTensor(),  # you can add other transformations in this list
#     transforms.Resize(size=(255, 255))
# ])

In [14]:
# all test data
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)
print(len(test_loader))

137


In [15]:
# for img_name, img in islice(enumerate(test_loader),0,2):
#     print(img_name, img)

## 3.Use a saved model for inference: (Predict for test data)
* [Stackoverflow: How do I predict using a PyTorch model?](https://stackoverflow.com/questions/66952664/how-do-i-predict-using-a-pytorch-model)

In [16]:
# all training data
# train_dataset = torchvision.datasets.ImageFolder(train_data_path, transform=transform, target_transform=None)
# print(train_dataset.class_to_idx)

# Save class dict (Use above data)
class_to_idx_dict = {'asparagus': 0, 'bambooshoots': 1, 'betel': 2, 'broccoli': 3, 'cauliflower': 4, 'chinesecabbage': 5, 'chinesechives': 6, 'custardapple': 7, 'grape': 8, 'greenhouse': 9, 'greenonion': 10, 'kale': 11, 'lemon': 12, 'lettuce': 13, 'litchi': 14, 'longan': 15, 'loofah': 16, 'mango': 17, 'onion': 18, 'others': 19, 'papaya': 20, 'passionfruit': 21, 'pear': 22, 'pennisetum': 23, 'redbeans': 24, 'roseapple': 25, 'sesbania': 26, 'soybeans': 27, 'sunhemp': 28, 'sweetpotato': 29, 'taro': 30, 'tea': 31, 'waterbamboo': 32}
class_to_idx_dict

{'asparagus': 0,
 'bambooshoots': 1,
 'betel': 2,
 'broccoli': 3,
 'cauliflower': 4,
 'chinesecabbage': 5,
 'chinesechives': 6,
 'custardapple': 7,
 'grape': 8,
 'greenhouse': 9,
 'greenonion': 10,
 'kale': 11,
 'lemon': 12,
 'lettuce': 13,
 'litchi': 14,
 'longan': 15,
 'loofah': 16,
 'mango': 17,
 'onion': 18,
 'others': 19,
 'papaya': 20,
 'passionfruit': 21,
 'pear': 22,
 'pennisetum': 23,
 'redbeans': 24,
 'roseapple': 25,
 'sesbania': 26,
 'soybeans': 27,
 'sunhemp': 28,
 'sweetpotato': 29,
 'taro': 30,
 'tea': 31,
 'waterbamboo': 32}

In [17]:
class_names_dict = {v: k for k, v in class_to_idx_dict.items()}  # Reverse/invert a dictionary mapping
print(class_names_dict)

{0: 'asparagus', 1: 'bambooshoots', 2: 'betel', 3: 'broccoli', 4: 'cauliflower', 5: 'chinesecabbage', 6: 'chinesechives', 7: 'custardapple', 8: 'grape', 9: 'greenhouse', 10: 'greenonion', 11: 'kale', 12: 'lemon', 13: 'lettuce', 14: 'litchi', 15: 'longan', 16: 'loofah', 17: 'mango', 18: 'onion', 19: 'others', 20: 'papaya', 21: 'passionfruit', 22: 'pear', 23: 'pennisetum', 24: 'redbeans', 25: 'roseapple', 26: 'sesbania', 27: 'soybeans', 28: 'sunhemp', 29: 'sweetpotato', 30: 'taro', 31: 'tea', 32: 'waterbamboo'}


In [18]:
# Ref: ALL_data_Model_1_efficientnet_訓練成功.ipynb

def test(model, test_loader, optimizer):  # predict
    model.eval()
    with torch.no_grad():
        ALL_img_name_list = []  # jpg檔名
        #ALL_pred_list = []  # 預測類別的index(上傳答案不用，不要算以減少運算資源)
        ALL_pred_class_str_list = []  # 預測類別index對應的字串
        for i, data in enumerate(tqdm(test_loader)):
        #for i, data in (islice(enumerate(tqdm(test_loader, 0)), 10)):
        #for i, data in tqdm(islice(enumerate(test_loader),0,2)):
        #for i, data in enumerate(test_loader, 0):
            img_name_list, x = data  # <class 'tuple'>, tensor
            x = x.cuda()
            optimizer.zero_grad()
            y_hat = model(x)  # tensor, torch.Size([10, 33])  
            pred_class_idx = torch.max(y_hat, 1)[1]
            #conf, classes = torch.max(y_hat, 1)  # tensor, tensor, 若要印conf
            #print(conf.item(), class_names_dict[classes.item()])
            
            pred_class_idx = pred_class_idx.detach().cpu().numpy()  # <class 'numpy.ndarray'>, (10, 1)
            ALL_img_name_list.extend(img_name_list)
            #pred_class_idx_list = pred_class_idx.tolist()  # list, 預測類別的index數值
            #ALL_pred_list.extend(pred_class_idx_list)
            pred_class_str = list(map(class_names_dict.get, pred_class_idx))  # list, 預測類別的字串種類
            ALL_pred_class_str_list.extend(pred_class_str)
            
        #print("ALL_img_name_list: ", ALL_img_name_list)
        #print("ALL_pred_list", ALL_pred_list)
        #print("ALL_pred_class_str_list", ALL_pred_class_str_list)
        return ALL_img_name_list, ALL_pred_class_str_list

In [19]:
test_flag = True
optimizer_conv = torch.optim.SGD(model_efficientnet.classifier[1].parameters(), lr=0.001, momentum=0.9)  # 只對最後一層參數做更新(指定最後一層:model_conv.fc)
optimizer = optimizer_conv

#如果test_flag=True,则加载已保存的模型
def use_save_model_to_predict(test_flag, save_model):
    if test_flag:
        model = save_model
        model.eval()
        ALL_img_name_list, ALL_pred_class_str_list = test(model, test_loader, optimizer)
    return ALL_img_name_list, ALL_pred_class_str_list
        
ALL_img_name_list, ALL_pred_class_str_list = use_save_model_to_predict(test_flag = True, save_model=model_efficientnet)

100%|██████████| 137/137 [03:32<00:00,  1.55s/it]


In [20]:
print(len(ALL_img_name_list))

1366


In [21]:
print(len(ALL_pred_class_str_list))

1366


In [22]:
prediction_df = pd.DataFrame(
    {'filename': ALL_img_name_list,
     'label': ALL_pred_class_str_list})

In [23]:
prediction_df.to_csv(output_path + "/efficientnet_preditions_folder5_6.csv", index=False)

In [24]:
print(output_path)

/content/gdrive/MyDrive/learning_AIdea2022/output
